In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
#import seaborn as sns
import pymongo
from pprint import pprint

In [2]:
client = pymongo.MongoClient()
db = client.metrics_causal

### What Experiments Do We Currently Have Data For?

#### CB

In [3]:
cb_collections = \
[str(col)
 for col 
 in sorted(db.collection_names())
 if 
     "CB_" in col and 
     "CR_" in col and 
     "VD" in col and
     "TEST" not in col and 
     "invalid" not in col.lower() and
 1==1]
cb_collections

['CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_FEATURE_SEL_VD',
 'CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_VD',
 'CR_CB_STACKED_VD',
 'CR_CB_TAGGING_VD_MOST_COMMON_TAG_RNN']

In [4]:
cb_collections_test = \
[str(col)
 for col 
 in sorted(db.collection_names())
 if 
     "CB_" in col and 
     "CR_" in col and 
     #"TD" not in col and    
     #"VD" not in col and
     "TEST" in col and 
     "invalid" not in col.lower() and
 1==1]
cb_collections_test

['TEST_CR_CB_STACKED_TD',
 'TEST_CR_CB_STACKED_VD',
 'TEST_CR_CB_TAGGING_TD_MOST_COMMON_TAG_RNN',
 'TEST_CR_CB_TAGGING_VD_MOST_COMMON_TAG_RNN']

[str(col)
 for col 
 in sorted(db.collection_names())
 if 
     "CB_" in col and 
     "CR_" in col and 
     "VD" in col and
     "TEST" in col and 
 1==1]

#### SC

In [5]:
sc_collections = \
[str(col)
 for col 
 in sorted(db.collection_names())
 if 
     "SC_" in col and 
     "CR_" in col and 
     "VD" in col and
     "TEST" not in col and 
     "invalid" not in col.lower() and
 1==1]
sc_collections

['CR_SC_STACKED_VD', 'CR_SC_TAGGING_VD_MOST_COMMON_TAG_RNN']

In [6]:
sc_collections_test = \
[str(col)
 for col 
 in sorted(db.collection_names())
 if 
     "SC_" in col and 
     "CR_" in col and 
     "TEST" in col and 
     "invalid" not in col.lower() and
 1==1]
sc_collections_test

['TEST_CR_SC_STACKED_TD',
 'TEST_CR_SC_STACKED_VD',
 'TEST_CR_SC_TAGGING_TD_MOST_COMMON_TAG_RNN',
 'TEST_CR_SC_TAGGING_VD_MOST_COMMON_TAG_RNN']

#### - The 'Stacked' and 'Most Common Tag' Are the Initial 2 Model Types (Tagging without Most Common was a Quick Experiment)

In [7]:
def compute_count_codes(row):

    keys = []
    unique = set()
    for k in row.keys():        
        if "->" in k: #k[0].isdigit():
            keys.append(k)
            unique.add(k)
    assert len(keys) == len(unique), "Duplicate keys found"
    return unique

In [8]:
sc_collections

['CR_SC_STACKED_VD', 'CR_SC_TAGGING_VD_MOST_COMMON_TAG_RNN']

In [9]:
def validate_collection(collection):
    
    col2codes = {}
    print("Rows by Collection:")
    for col in collection:
        cursor = db[col].find({})
        print(col, cursor.count())
        first_row_codes = None
        for row in cursor:
            codes = compute_count_codes(row)
            if not first_row_codes:
                first_row_codes = set(codes)
                col2codes[col] = codes
            else:            
                assert first_row_codes == codes, "Different codes computed across different runs"

    print("\nCodes by Collection:")
    for col,codes in col2codes.items():
        print(col, len(codes))

    print("\nDifferent Counts:")
    for col, codes in col2codes.items():
        for col2, codes2 in col2codes.items():
            if col >= col2:
                continue

            if codes != codes2:
                diff_left  = codes - codes2
                diff_right = codes2 - codes

                print("Different codes computed for collections: 1: {col1} [{count_1}] and 2: {col2} [{count_2}]"\
                      .format(col1=col, count_1=len(codes), col2=col2, count_2=len(codes2)))

                print("Left Count: {lc} Right Count: {rc}".format(lc=len(diff_left), rc=len(diff_right)))
                print("\tLEFT:  {left}\n\tRIGHT: {right}".format(left="\t".join(diff_left), right="\t".join(diff_right)))
                print("")

In [10]:
validate_collection(cb_collections)

Rows by Collection:
CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_FEATURE_SEL_VD 124
CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_VD 1
CR_CB_STACKED_VD 49
CR_CB_TAGGING_VD_MOST_COMMON_TAG_RNN 6

Codes by Collection:
CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_FEATURE_SEL_VD 86
CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_VD 86
CR_CB_STACKED_VD 86
CR_CB_TAGGING_VD_MOST_COMMON_TAG_RNN 86

Different Counts:


In [11]:
validate_collection(cb_collections_test)

Rows by Collection:
TEST_CR_CB_STACKED_TD 1
TEST_CR_CB_STACKED_VD 1
TEST_CR_CB_TAGGING_TD_MOST_COMMON_TAG_RNN 1
TEST_CR_CB_TAGGING_VD_MOST_COMMON_TAG_RNN 1

Codes by Collection:
TEST_CR_CB_STACKED_TD 86
TEST_CR_CB_STACKED_VD 86
TEST_CR_CB_TAGGING_TD_MOST_COMMON_TAG_RNN 86
TEST_CR_CB_TAGGING_VD_MOST_COMMON_TAG_RNN 86

Different Counts:


In [12]:
#Train and Validation Counts should match
#CB_COLLECTION_TD = [col.replace("VD","TD") for col in CB_COLLECTION]
#validate_collection(CB_COLLECTION_TD)

In [13]:
validate_collection(sc_collections)

Rows by Collection:
CR_SC_STACKED_VD 49
CR_SC_TAGGING_VD_MOST_COMMON_TAG_RNN 6

Codes by Collection:
CR_SC_STACKED_VD 49
CR_SC_TAGGING_VD_MOST_COMMON_TAG_RNN 49

Different Counts:


In [14]:
validate_collection(sc_collections_test)

Rows by Collection:
TEST_CR_SC_STACKED_TD 1
TEST_CR_SC_STACKED_VD 1
TEST_CR_SC_TAGGING_TD_MOST_COMMON_TAG_RNN 1
TEST_CR_SC_TAGGING_VD_MOST_COMMON_TAG_RNN 1

Codes by Collection:
TEST_CR_SC_STACKED_TD 49
TEST_CR_SC_STACKED_VD 49
TEST_CR_SC_TAGGING_TD_MOST_COMMON_TAG_RNN 49
TEST_CR_SC_TAGGING_VD_MOST_COMMON_TAG_RNN 49

Different Counts:
